In [1]:
import nest_asyncio
nest_asyncio.apply()

import json
import traceback
import sys
sys.path.append("../")

from loguru import logger
from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.outputs import Generation
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import StrOutputParser
from typing import List, Union, Any, Dict
from langchain_core.pydantic_v1 import BaseModel, Field

from desci_sense.shared_functions.configs import MultiParserChainConfig, MultiRefTaggerChainConfig, MetadataExtractionConfig, LLMConfig, ParserChainType, RefTaggerChainConfig
from desci_sense.shared_functions.init import init_multi_chain_parser_config
from desci_sense.shared_functions.parsers.multi_chain_parser import MultiChainParser
from desci_sense.shared_functions.parsers.multi_reference_tagger import normalize_labels, normalize_references
from desci_sense.shared_functions.web_extractors.metadata_extractors import extract_posts_ref_metadata_dict, RefMetadata
from desci_sense.shared_functions.dataloaders import (
    scrape_post,
    convert_text_to_ref_post,
)
from desci_sense.shared_functions.utils import _find_json_object
from desci_sense.shared_functions.postprocessing import ParserChainOutput

/Users/ronentamari/anaconda3/envs/asensebot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
multi_config = MultiParserChainConfig(
        parser_configs=[
            MultiRefTaggerChainConfig(name="multi_ref_tagger",
                                      llm_config=LLMConfig(llm_type="mistralai/mixtral-8x7b-instruct:nitro")),
                                      RefTaggerChainConfig(name="ref_tagger",
                                      llm_config=LLMConfig(llm_type="mistralai/mixtral-8x7b-instruct:nitro")),
        ],
        post_process_type="combined",
        metadata_extract_config=MetadataExtractionConfig(extraction_method="citoid")
    )

# mistralai/mixtral-8x7b-instruct:nitro
# openai/gpt-3.5-turbo

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (326804237.py, line 7)

In [3]:
multi_chain_parser = MultiChainParser(multi_config)

2024-05-03 14:40:02.692 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:66 - Initializing MultiChainParser. PostProcessType=none
2024-05-03 14:40:02.699 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:73 - Initializing post parsers...
2024-05-03 14:40:02.700 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'multi_ref_tagger' 
2024-05-03 14:40:02.768 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'ref_tagger' 


In [12]:
multi_chain_parser.pparsers["multi_ref_tagger"].parser_config

KeywordPParserChainConfig(name='multi_ref_tagger', type=<ParserChainType.MULTI_REF_TAGGER: 'multi_reference_tagger'>, llm_config=LLMConfig(llm_type='mistralai/mixtral-8x7b-instruct:nitro', temperature='0.6'), use_metadata=True, max_keywords=6)

In [4]:
TEST_POST_TEXT_W_NO_REFS = """
These 2 papers are highly recommended!
"""
batch = [convert_text_to_ref_post(TEST_POST_TEXT_W_NO_REFS) for _ in range(6)]
# post_nr = convert_text_to_ref_post(TEST_POST_TEXT_W_NO_REFS)
# md_dict = extract_posts_ref_metadata_dict([post_nr], md_type=multi_config.metadata_extract_config.extraction_method)

In [7]:
res = multi_chain_parser.batch_process_ref_posts(batch, batch_size=6)

2024-05-01 17:03:30.984 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:232 - Processing 6 posts with parsers: ['multi_ref_tagger']
2024-05-01 17:03:30.984 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:234 - Instantiating prompts...
100%|██████████| 6/6 [00:07<00:00,  1.28s/it]
2024-05-01 17:03:38.636 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:256 - Post processing 6 results...
2024-05-01 17:03:38.637 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:268 - Done!


In [9]:
for i in range(len(res)):
    print(res[i]['multi_ref_tagger'].answer.sub_answers[0].final_answer)

['missing-ref']
['missing-ref']
['missing-ref']
['missing-ref']
['missing-ref']
['missing-ref']


In [5]:
res = multi_chain_parser.process_text(TEST_POST_TEXT_W_NO_REFS)

2024-05-03 14:40:26.256 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:185 - Processing post with parsers: ['multi_ref_tagger', 'ref_tagger']
2024-05-03 14:40:26.257 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:187 - Instantiating prompts...
2024-05-03 14:40:26.258 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:192 - Invoking parallel chain...


In [10]:
answer = res["multi_ref_tagger"].answer
answer

Answer(sub_answers=[SubAnswer(ref_number=0, reasoning_steps='The post is recommending two scientific papers, but it does not provide a link to them, so the <missing-ref> tag is appropriate.', candidate_tags=['<missing-ref>', '<dg-observation>', '<reading>'], final_answer=['missing-ref'], ref_url=None)], debug={})

In [13]:
parser = multi_chain_parser.pparsers["multi_ref_tagger"]

In [10]:
post = convert_text_to_ref_post(TEST_POST_TEXT_W_NO_REFS)
md_dict = extract_posts_ref_metadata_dict([post], md_type=multi_config.metadata_extract_config.extraction_method)
prompt = parser.instantiate_prompt(post, md_dict)

In [12]:
res = parser.chat(prompt["multi_ref_tagger_input"])

In [13]:
res

' Here\'s the structured semantic format for the given social media post:\n\n```json\n{\n  "sub_answers": [\n    {\n      "reasoning_steps": "The post recommends two papers, but does not provide a URL link to them. Therefore, it does not fall under the <missing-ref> tag.",\n      "candidate_tags": [],\n      "final_answer": ["<reading>"]\n    }\n  ]\n}\n```\n\nThe post recommends two papers, which means the author has either read them in the past, is reading them in the present, or is looking forward to reading them in the future. Therefore, the <reading> tag is the most suitable tag for this post.'

In [17]:
_find_json_object(res)

'{\n  "sub_answers": [\n    {\n      "reasoning_steps": "The post recommends two papers, but does not provide a URL link to them. Therefore, it does not fall under the <missing-ref> tag.",\n      "candidate_tags": [],\n      "final_answer": ["<reading>"]\n    }\n  ]\n}'

In [14]:
parser.pydantic_parser.parse(None)

ValidationError: 1 validation error for Generation
text
  none is not an allowed value (type=type_error.none.not_allowed)

In [ ]:
answer = res["multi_ref_tagger"].answer
res["multi_ref_tagger"].extra

{'prompt': '\nYou are an expert annotator tasked with converting social media posts about scientific research to a structured semantic format. For an input post, your job is to select the tags most suitable to that post, from a predefined set of tags. \n\n  The available tag types are:\n  <missing-ref>: this post seems to be referring to a reference by name but has not explicitly provided a URL link to the reference. For example, a post that discusses a book and mentions it by title, but contains no link to the book.\n  <dg-question>: this post is raising a research question.\n  <listening>: this post describes the listening status of the author in relation to a reference, such as a podcast or radio station. The author may have listened to the content in the past, is listening to the content in the present, or is looking forward to listening the content in the future.\n  <call-for-papers>: this post contains a call for research papers, for example to a journal, conference or workshop.\

In [ ]:
answer

Answer(sub_answers=[SubAnswer(ref_number=0, reasoning_steps='The post is recommending 2 scientific papers, but is not providing direct links to them. I will tag this as <missing-ref> because it seems to be referring to references by name, but has not explicitly provided URL links to the references.', candidate_tags=['<missing-ref>', '<dg-observation>', '<reading>'], final_answer=['missing-ref'], ref_url=None)], debug={})

In [ ]:
res["multi_ref_tagger"].answer.sub_answers[0].final_answer

['dg-claim', 'reading']

In [7]:
TEST_POST_TEXT_W_2_REFS = """
These 2 papers are highly recommended!
https://arxiv.org/abs/2402.04607
https://royalsocietypublishing.org/doi/10.1098/rstb.2022.0267
"""
# post_2 = convert_text_to_ref_post(TEST_POST_TEXT_W_2_REFS)
# md_dict = extract_posts_ref_metadata_dict([post_2], md_type=multi_config.metadata_extract_config.extraction_method)

In [6]:
res = multi_chain_parser.process_text(TEST_POST_TEXT_W_2_REFS)

2024-05-01 09:38:34.876 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async:20 - target_url=https://arxiv.org/abs/2402.04607
2024-05-01 09:38:34.898 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async:20 - target_url=https://royalsocietypublishing.org/doi/10.1098/rstb.2022.0267
2024-05-01 09:38:36.977 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:184 - Processing post with parsers: ['multi_ref_tagger']
2024-05-01 09:38:36.979 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:186 - Instantiating prompts...
2024-05-01 09:38:36.981 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:191 - Invoking parallel chain...


In [8]:
print(res["multi_ref_tagger"].answer.sub_answers[0].final_answer)

['recommendation', 'reading']


In [9]:
print(res["multi_ref_tagger"].answer.sub_answers[0].ref_url)

https://arxiv.org/abs/2402.04607


In [10]:
print(res["multi_ref_tagger"].answer.sub_answers[1].final_answer)

['recommendation', 'reading']


In [8]:
answer = res["multi_ref_tagger"].reasoning
answer

{0: 'The post recommends this preprint, which suggests that Google Scholar is manipulatable through citation fraud. The author of the post likely agrees with the findings of the preprint and finds it interesting or important.',
 1: 'The post recommends this journal article, which presents a model of group selection with a global externality. The author of the post likely finds the article interesting or important, but there is no explicit agreement or disagreement expressed regarding its contents.'}

In [7]:
answer = res["multi_ref_tagger"].answer

In [8]:
answer.to_combined_format()

[['recommendation'], ['recommendation']]

In [13]:
sub_answers = answer.sub_answers
sub_answers

[SubAnswer(ref_number=0, reasoning_steps="The post recommends this preprint titled 'Google Scholar is manipulatable'.", candidate_tags='[<recommendation>, <reading>, <announce>]', final_answer=['recommendation', 'reading'], ref_url='https://arxiv.org/abs/2402.04607'),
 SubAnswer(ref_number=1, reasoning_steps="The post recommends this journal article titled 'Reducing global inequality increases local cooperation: a simple model of group selection with a global externality'.", candidate_tags='[<recommendation>, <reading>, <announce>]', final_answer=['recommendation', 'reading'], ref_url='https://royalsocietypublishing.org/doi/10.1098/rstb.2022.0267')]

In [14]:
sorted(sub_answers, key=lambda x: x.ref_number)

[SubAnswer(ref_number=0, reasoning_steps="The post recommends this preprint titled 'Google Scholar is manipulatable'.", candidate_tags='[<recommendation>, <reading>, <announce>]', final_answer=['recommendation', 'reading'], ref_url='https://arxiv.org/abs/2402.04607'),
 SubAnswer(ref_number=1, reasoning_steps="The post recommends this journal article titled 'Reducing global inequality increases local cooperation: a simple model of group selection with a global externality'.", candidate_tags='[<recommendation>, <reading>, <announce>]', final_answer=['recommendation', 'reading'], ref_url='https://royalsocietypublishing.org/doi/10.1098/rstb.2022.0267')]

In [2]:
multi_config = MultiParserChainConfig(
        parser_configs=[
            RefTaggerChainConfig(name="ref_tagger",
                                      llm_config=LLMConfig(llm_type="mistralai/mixtral-8x7b-instruct:nitro"))
        ],
        metadata_extract_config=MetadataExtractionConfig(extraction_method="citoid")
    )
multi_chain_parser = MultiChainParser(multi_config)


2024-05-01 18:22:32.929 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:66 - Initializing MultiChainParser. PostProcessType=none
2024-05-01 18:22:32.932 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:73 - Initializing post parsers...
2024-05-01 18:22:32.932 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'ref_tagger' 


In [5]:
res = multi_chain_parser.process_text(TEST_POST_TEXT_W_NO_REFS)

2024-05-01 18:22:49.330 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:185 - Processing post with parsers: ['ref_tagger']
2024-05-01 18:22:49.331 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:187 - Instantiating prompts...
2024-05-01 18:22:49.332 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:192 - Invoking parallel chain...


In [6]:
res

{'ref_tagger': ParserChainOutput(answer=[['missing-ref', 'reading']], pparser_type=<ParserChainType.REFERENCE_TAGGER: 'reference_tagger'>, reasoning='[Reasoning Steps]\n\n1. The author is recommending two papers, which implies that they have read or are familiar with the content of these papers.\n2. The author does not provide explicit URL links to these papers, but they are clearly identified by name.\n\n[Candidate Tags]\n\n- <reading>: The author has likely read or is familiar with the content of the two papers they are recommending.\n- <missing-ref>: The author does not provide explicit URL links to the two papers they are recommending, but they are identified by name.', extra={'allowed_tags': ['missing-ref', 'dg-question', 'listening', 'call-for-papers', 'dg-observation', 'dg-claim', 'funding', 'watching', 'reading', 'event', 'job'], 'full_text': " Reasoning Steps:\n1. The author is recommending two papers, which implies that they have read or are familiar with the content of these

In [8]:
res = multi_chain_parser.process_text(TEST_POST_TEXT_W_2_REFS)

2024-05-01 18:23:25.734 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:54 - Retry attempt 1
2024-05-01 18:23:25.734 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:76 - target_url=https://arxiv.org/abs/2402.04607
2024-05-01 18:23:25.748 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:54 - Retry attempt 1
2024-05-01 18:23:25.748 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:76 - target_url=https://royalsocietypublishing.org/doi/10.1098/rstb.2022.0267
2024-05-01 18:23:27.938 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:185 - Processing post with parsers: ['ref_tagger']
2024-05-01 18:23:27.939 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:187 - Instantiating prompts...
2024-05-01 18:23:27.940 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:proc

In [9]:
res

{'ref_tagger': ParserChainOutput(answer=[['recommendation'], ['recommendation']], pparser_type=<ParserChainType.REFERENCE_TAGGER: 'reference_tagger'>, reasoning='[Reasoning Steps]\n\n1. The author of the post is recommending two scientific papers, indicating a <recommendation> tag.\n2. The post does not contain any disagreement or agreement with the papers, so no <disagrees> or <agrees> tags are applicable.\n3. The post does not contain a review, announcement, or discussion of the papers, so no <review>, <announce>, or <discussion> tags are applicable.\n4. The post does not indicate that the author is listening to, watching, or funding the papers, so no <listening>, <watching>, or <funding> tags are applicable.\n5. The post does not contain a question about the papers, so no <question> tag is applicable.\n6. The post does not mention an event, job, or indicate interest using the 👀 emoji, so no <event>, <job>, or <indicates-interest> tags are applicable.\n\n[Candidate Tags]\n\n* <recomm